In [0]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 4.2MB/s 


In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [0]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

In [0]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [0]:
class ClearTrainingOutput(keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait=True)

In [12]:
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test), callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [13]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4716 - accuracy: 0.8310 - val_loss: 0.4229 - val_accuracy: 0.8416
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3560 - accuracy: 0.8703 - val_loss: 0.3975 - val_accuracy: 0.8536
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3227 - accuracy: 0.8819 - val_loss: 0.3729 - val_accuracy: 0.8652
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3006 - accuracy: 0.8882 - val_loss: 0.3390 - val_accuracy: 0.8775
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2798 - accuracy: 0.8964 - val_loss: 0.3338 - val_accuracy: 0.8799
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2643 - accuracy: 0.9015 - val_loss: 0.3488 - val_accuracy: 0.8759
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2525 - accuracy: 0.9047 - val_loss: 0.3292 - val_accuracy: